In [1]:
# !pip install youtube-transcript-api
# !pip install openai
# !pip install --upgrade openai
# !pip install dotenv

In [14]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import JSONFormatter
from pytube import YouTube

import json
import openai

from openai import OpenAI
client = OpenAI()

import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
DEFAULT_YOUTUBE_URL = "https://youtube.com/watch?v="

In [15]:
import openai
from packaging import version

required_version = version.parse("1.1.1")
current_version = version.parse(openai.__version__)

print(f"OpenAI version is {current_version}.")

if current_version < required_version:
    raise ValueError(f"Error: OpenAI version {openai.__version__}"
                     " is less than the required version 1.1.1")
else:
    print("OpenAI version is compatible.")

OpenAI version is 1.11.1.
OpenAI version is compatible.


In [20]:
print(openai.model);

def valid_openai_api_key(api_key):
    openai.api_key = api_key;
    
    try:
        models = openai.models;
        for idx, model in enumerate(models['data']):
            print("{}: {} owned_by {}".format(idx, model['id'], model['owned_by']))
    except openai.error.AuthenticationError as e:
        return False
    else:
        return True

print("")

if valid_openai_api_key(OPENAI_API_KEY):
    print("Valid OpenAI API key.")
else:
    print("Invalid OpenAI API key.")

AttributeError: module 'openai' has no attribute 'model'

In [21]:
def youtube_transcript_to_json(video_id):
    # Must be a single transcript.
    transcript = YouTubeTranscriptApi.get_transcript(video_id, preserve_formatting=True)
    
    formatter = JSONFormatter()
    # .format_transcript(transcript) turns the transcript into a JSON string.
    json_formatted = formatter.format_transcript(transcript)

    yt = YouTube(DEFAULT_YOUTUBE_URL + video_id)
    # Now we can write it out to a file.
    with open(yt.streams[0].title + '.json', 'w', encoding='utf-8') as json_file:
        json_file.write(json_formatted)

    # Now should have a new JSON file that you can easily read back into Python.
    json_object = json.loads(json_formatted)
    
    return json_object

In [47]:
# formatter = JSONFormatter()
# json_formatted = JSONFormatter().format_transcript(transcript_list, indent=2)

In [23]:
json_object = youtube_transcript_to_json('ZZ_vnqvW4DQ')

print(json.dumps(json_object, indent=2))

[
  {
    "text": "What is Agile Methodology?",
    "start": 1.699,
    "duration": 1.0
  },
  {
    "text": "Agile is a set of values and principles yet\npeople are constantly asking about \"Agile",
    "start": 2.699,
    "duration": 4.651
  },
  {
    "text": "Methodology.\"",
    "start": 7.35,
    "duration": 1.0
  },
  {
    "text": "If you want to know more about agile values\nand principles, take a look at my video called",
    "start": 8.35,
    "duration": 3.76
  },
  {
    "text": "\"What is Agile?\"",
    "start": 12.11,
    "duration": 1.18
  },
  {
    "text": "In this video we are going to try to answer\n\"What is Agile Methodology?\"",
    "start": 13.29,
    "duration": 5.48
  },
  {
    "text": "This is a bit of a trick question.",
    "start": 18.77,
    "duration": 1.41
  },
  {
    "text": "A methodology is a body of methods, procedures,\nand rules for a particular discipline.",
    "start": 20.18,
    "duration": 4.89
  },
  {
    "text": "But Agile values and pri

In [6]:
# json_object = youtube_transcript_to_json('2TL3DgIMY1g')

# print(json.dumps(youtube_transcript_to_json('2TL3DgIMY1g'), indent=2))

In [25]:
output = ''
for x in json_object:
    sentence = x['text']
    output += f' {sentence}\n'
  
# print(len(output))
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    response_format={ "type": "json_object" },
    messages=[
        {"role": "system", "content": "You are a journalist."},
        {"role": "assistant", "content": "output a list of tags for this blog post in a python list such as ['tag1', 'tag2', 'tag3']"},
        {"role": "user", "content": output}
    ]
)

print(response.choices[0].message.content)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}